In [2]:
# import aedat
import os 
import pandas as pd 
# import aer 
import torch 
import spikingjelly.datasets as sjds 

In [3]:
path_to_dataset = "/home/rlwagun/Files/datasets/DVS_Gesture_dataset/DvsGesture"


gesture_mapping_file = os.path.join(path_to_dataset, "gesture_mapping.csv")
# gesture mapping label and action # starts from 1 to 11 
gesture_mapping = pd.read_csv(gesture_mapping_file)
gesture_mapping_dict = dict(zip(gesture_mapping['label'], gesture_mapping['action']))

user_name = "user01_fluorescent"
user_dvs_data = os.path.join(path_to_dataset, user_name + ".aedat")
user_dvs_labels = os.path.join(path_to_dataset, user_name + "_labels.csv")
# decoder = aedat.Decoder(dvs_data)
# print(decoder.id_to_stream())

raw_events = sjds.load_aedat_v3(user_dvs_data)

In [30]:
gesture_mapping

,action,label
0,hand_clapping,1
1,right_hand_wave,2
2,left_hand_wave,3
3,right_arm_clockwise,4
4,right_arm_counter_clockwise,5
5,left_arm_clockwise,6
6,left_arm_counter_clockwise,7
7,arm_roll,8
8,air_drums,9
9,air_guitar,10


In [4]:
labels = pd.read_csv(user_dvs_labels)
# setting the event as the class index 
labels.set_index("class", inplace=True) 


In [5]:
import glob
import numpy as np 
paths = glob.glob(path_to_dataset + "/*_labels.csv")

In [6]:
list(map(lambda x: len(raw_events[x]), ['t', 'x', 'y', 'p'] ))

[7790918, 7790918, 7790918, 7790918]

In [7]:
events = [] 

for t, x, y, p in zip(raw_events['t'], raw_events['x'], raw_events['y'], raw_events['p']):
    events.append([t, x, y])

events = np.array(events) 

In [8]:
df = pd.DataFrame(raw_events)

In [31]:
labels

,startTime_usec,endTime_usec
class,,
1,80048239,85092709
2,89431170,95231007
3,95938861,103200075
4,114845417,123499505
5,124344363,131742581
6,133660637,141880879
7,142360393,149138239
8,150717639,157362334
8,157773346,164029864


In [36]:
user_class = 4
filtered_df = df.loc[(df['t'] >= labels.iloc[user_class]["startTime_usec"]) & (df['t'] <= labels.iloc[user_class]["endTime_usec"])]

In [37]:
filtered_df

,t,x,y,p
2410336,124344371,78,126,1
2410337,124344375,56,102,0
2410338,124344388,46,102,1
2410339,124344436,52,101,0
2410340,124344461,55,104,1
...,...,...,...,...
3269142,131742521,40,117,1
3269143,131742561,117,108,0
3269144,131742568,48,97,1
3269145,131742569,32,82,1


In [38]:
filtered_events = filtered_df.drop(columns=['p']).to_numpy()
filtered_events

array([[124344371,        78,       126],
       [124344375,        56,       102],
       [124344388,        46,       102],
       ...,
       [131742568,        48,        97],
       [131742569,        32,        82],
       [131742575,        29,        48]])

In [39]:
import e2v
import cv2 
for frame in e2v.get_event_frame(filtered_events, 
    threshold=0.8,
    decay=0.01,
    image_scale=10):
    cv2.imshow("test", cv2.resize(frame, np.array(frame.shape[::-1]) * 5))
    k = cv2.waitKey(1)
    if k == ord('q'):
        break 
cv2.destroyAllWindows()


In [84]:
labels.iloc[0]["startTime_usec"], labels.iloc[0]["endTime_useca"]


(np.int64(80048239), np.int64(85092709))

In [86]:
sub_event = events[(events >= labels.iloc[0]["startTime_usec"]) & (events <= labels.iloc[0]["endTime_usec"])]

array([80048267, 80048277, 80048278, ..., 85092406, 85092538, 85092700])